In [1]:
import pandas as pd

In [2]:
commits = pd.read_pickle('../../data/01_data_from_db/commits.pkl')
commits.head()

,commit_id,committer_id,project_id,created_at
0,1,17,10,2012-04-08 02:29:18
1,2,18,10,2011-12-29 21:12:27
2,3,18,10,2011-12-29 21:11:10
3,4,17,10,2011-12-29 20:54:15
4,5,18,10,2011-12-29 20:29:49


In [3]:
commits['year'] = commits.created_at.dt.year
commits['month'] = commits.created_at.dt.month
commits.head()

,commit_id,committer_id,project_id,created_at,year,month
0,1,17,10,2012-04-08 02:29:18,2012,4
1,2,18,10,2011-12-29 21:12:27,2011,12
2,3,18,10,2011-12-29 21:11:10,2011,12
3,4,17,10,2011-12-29 20:54:15,2011,12
4,5,18,10,2011-12-29 20:29:49,2011,12


## Nowe commity w miesiącu

In [4]:
new_commits = commits.groupby(['project_id', 'year', 'month']).count().reset_index().rename(columns={'created_at': 'new_commits'}).drop(columns={'commit_id', 'committer_id'}, axis=1)
new_commits

,project_id,year,month,new_commits
0,2,2010,5,43
1,2,2010,6,4
2,2,2010,7,3
3,2,2010,8,1
4,2,2010,9,1
...,...,...,...,...
18757,90343,2013,9,2
18758,90366,2013,9,13
18759,91413,2013,9,1
18760,91417,2013,9,1


Wersja alternatywna

In [5]:
commits.groupby(by= ['project_id', 'year', 'month'], as_index=False).agg({'created_at': pd.Series.nunique}).rename(columns={'created_at': 'new_commits'})

,project_id,year,month,new_commits
0,2,2010,5,43
1,2,2010,6,4
2,2,2010,7,3
3,2,2010,8,1
4,2,2010,9,1
...,...,...,...,...
18757,90343,2013,9,2
18758,90366,2013,9,13
18759,91413,2013,9,1
18760,91417,2013,9,1


## Dołaczenie łącznej liczby commitów od początku projektu (uwaga - długie wykonanie)

In [6]:
total_commits_list = []
for index, row in new_commits.iterrows():
    total_commits_list.append(new_commits[(((row['year'] > new_commits.year) | ((row['year'] == new_commits.year) & (row['month'] > new_commits.month))) & (row['project_id'] == new_commits.project_id))].new_commits.sum())

In [7]:
new_commits['total_commits'] = pd.Series(total_commits_list)

Sprawdzenie dla jednego z projektów

In [8]:
new_commits[new_commits.project_id == 2]

,project_id,year,month,new_commits,total_commits
0,2,2010,5,43,0
1,2,2010,6,4,43
2,2,2010,7,3,47
3,2,2010,8,1,50
4,2,2010,9,1,51
5,2,2010,10,3,52
6,2,2010,11,9,55
7,2,2011,1,1,64
8,2,2011,5,4,65
9,2,2011,6,74,69


In [9]:
new_commits.to_pickle('../../data/02_prepared_data/new_commits.pkl')

## Liczba unikalnych commitujących w miesiącu

In [10]:
new_unique_committers = commits.groupby(by = ['project_id', 'year', 'month'], as_index=False).agg({'committer_id': pd.Series.nunique}).rename(columns={'committer_id': 'new_unique_committers'})
new_unique_committers

,project_id,year,month,new_unique_committers
0,2,2010,5,1
1,2,2010,6,1
2,2,2010,7,1
3,2,2010,8,1
4,2,2010,9,1
...,...,...,...,...
18757,90343,2013,9,1
18758,90366,2013,9,1
18759,91413,2013,9,1
18760,91417,2013,9,1


In [11]:
new_unique_committers.to_pickle('../../data/02_prepared_data/new_unique_committers.pkl')

## (tmp) Dołączenie nowych commitów w miesiącu do tablicy projektów

> **TODO** TRZEBA ZASTANOWIĆ SIĘ, CO ZROBIĆ Z COMMITAMI, KTÓRE POWSTAŁY PRZED UTWORZENIEM PROJEKTÓW

In [12]:
# projects = pd.read_pickle('../../data/01_data_from_db/projects.pkl')

In [13]:
# with_new_commits = pd.merge(projects, new_commits, on='project_id', how='left', sort=False)
# with_new_commits.shape

In [14]:
# with_new_commits.isnull().sum()

In [15]:
# with_new_commits.dropna(how='any').shape

In [16]:
# projects.sort_values(by=['created_at'])

In [17]:
# commits.sort_values(by=['created_at'])

In [18]:
# for y in range(2003, 2015):
#     print(f'{y}: {commits[commits.year == y].year.count().sum()}')